# 1D CNN for 12-Lead ECG Classification

**Main Thesis Topic:** “Zero-shot classification of ECG signals using CLIP-like model”. 

**For example:** Train on PBT-XL: 

- **Text Encoder:** ClinicalBERT
- **Image Encoder:** 1D CNN

**Experiment A):** Baseline: We can take only the name of the class. For example, take “Myocardial Infarction” as a text. We should exclude some classes from training and after training is completed, the CLIP-like model can be tested on these excluded classes. 
Next, we get embeddings of text from biobert and train ECG encoder with contrastive loss.

**Experiment B):** Same as Experiment A but instead of testing on the same dataset/classes, we would test on other datasets containing different classes.

**Evaluation metrics:** 
- Main: AUC-ROC, average_precison_score, 
- Optional: Specificity, Sensitivity, F1-score 

Outcome: 

It’s possible to train CLIP-like models with freezed (or unchanged/not fine tuned for downstream tasks) text encoder
Training ECG encoders that are viable for representing different domains (within ECG modality) and previously unseen classes. 
Training a CLIP-like model on ECGs has little novelty.

https://www.baeldung.com/cs/contrastive-learning

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html



In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import sys

In [ ]:
sys.path.append('C:/Users/navme/Desktop/ECG_Project/PyFiles')

In [ ]:
# Import helper functions
import helper_functions as hf
import helper_function_tests as hft
from dataset import PhysioNetDataset

In [ ]:
PhysioNet_PATH = f'C:/Users/navme/Desktop/ECG_Thesis_Local/PhysioNet-2021-Challenge/physionet.org/files/challenge-2021/1.0.3/training'
PhysioNet_PATH

In [ ]:
train_set = PhysioNetDataset(PhysioNet_PATH, train = True)
test_set = PhysioNetDataset(PhysioNet_PATH, train = False)

print("There are {0} datasets in the train_set and {1} datasets in test_set".format(len(train_set), len(test_set)))

## SNOWMED-CT Codes

In [5]:
import requests

# Set the base URL for the SNOMED International's REST API
base_url = 'https://browser.ihtsdotools.org/snowstorm/snomed-ct'

# Set the search term
search_term = 'asthma'

# Set the query parameters
query_params = {
    'term': search_term,
    'activeFilter': 'true',
    'conceptActive': 'true',
    'semanticFilter': 'disorder',
    'termActive': 'true',
    'lang': 'english',
    'skipTo': '0',
    'returnLimit': '50'
}

try:
    # Send the GET request to the SNOMED International's REST API
    response = requests.get(f'{base_url}/descriptions', params=query_params)

    # Raise an exception if the response status code is not 200
    response.raise_for_status()

    # Parse the response JSON
    response_json = response.json()

    # Extract the results
    results = response_json['items']

    # Print the results
    for result in results:
        print(result['concept']['conceptId'], result['term'])

except requests.exceptions.HTTPError as err:
    print(f'HTTP error occurred: {err}')
except requests.exceptions.RequestException as err:
    print(f'An error occurred: {err}')

HTTP error occurred: 423 Client Error:  for url: https://browser.ihtsdotools.org/snowstorm/snomed-ct/descriptions?term=asthma&activeFilter=true&conceptActive=true&semanticFilter=disorder&termActive=true&lang=english&skipTo=0&returnLimit=50


## Model